In [30]:
import json
import yaml
import glob
import os
import numpy as np
import sys
import shutil
import time

import sys
sys.path.insert(0, "../../hindsight/hindsight_server/")

from utils import make_dir

In [6]:
project = "twitter-2024-09-13-20-23-70120e04_scrolling_entity"
project_dir = os.path.abspath(f"../data/label_studio/{project}")

In [7]:
# Create train.config.yaml
config_d = {"path" : project_dir, "train" : "images", "val" : "images"}
with open(os.path.join(project_dir, "notes.json"), 'r') as infile:
    project_notes = json.load(infile)

id_to_entity = {}
for cat in project_notes['categories']:
    id_to_entity[cat['id']] = cat['name']

# If you want to create new label mappings

In [8]:
training_mapping = {'discover_new_communities' : "scrolling_entity", "partial_tweet" : "scrolling_entity", "pinned_by_people_you_follow" : "scrolling_entity",
                    "tweet" : "scrolling_entity", "tweet_ad": "scrolling_entity"}

In [9]:
new_id_to_entity = {}
for i, entity in enumerate(set(training_mapping.values())):
    new_id_to_entity[i] = entity

In [10]:
entity_to_id = {v : k for k, v in id_to_entity.items()}
new_entity_to_id = {v : k for k, v in new_id_to_entity.items()}

old_id_to_new_id = {}
for old_entity, new_entity in training_mapping.items():
    old_id_to_new_id[entity_to_id[old_entity]] = new_entity_to_id[new_entity]

In [12]:
config_d["names"] = new_id_to_entity

config_f = os.path.join(project_dir, "train_config.yaml")
with open(config_f, 'w') as outfile:
    yaml.dump(config_d, outfile)

# Modify labels to accomodate remapping

In [14]:
labels_dir = os.path.join(project_dir, "labels")
org_labels_dir = os.path.join(project_dir, "labels_org")
if not os.path.exists(org_labels_dir):
    shutil.copytree(labels_dir, org_labels_dir)

In [15]:
full_width_entities = {"scrolling_entity"}
full_width_ids = {new_entity_to_id[e] for e in full_width_entities}

In [17]:
for label_f in os.listdir(org_labels_dir):
    new_label_data = list()
    with open(os.path.join(org_labels_dir, label_f), 'r') as infile:
        for line in infile.readlines():
            line_s = line.split(" ")
            entity_id = int(line_s[0])
            if entity_id in old_id_to_new_id:
                entity_id = old_id_to_new_id[entity_id]
                
                if entity_id in full_width_ids:
                    line = " ".join([str(entity_id), str(0.5), line_s[2], str(1), line_s[4]])
                else:
                    line = " ".join([str(entity_id), line_s[1], line_s[2], line_s[3], line_s[4]])
        
                new_label_data.append(line)
    
    new_label_f = os.path.join(labels_dir, label_f)
    with open(new_label_f, 'w') as outfile:
        for new_line in new_label_data:
            outfile.write(new_line + "\n")

# Run Training

In [18]:
import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

In [19]:
from ultralytics import YOLO

In [20]:
model = YOLO("yolov8m.pt")

In [31]:
current_time = int(time.time())
model_run_dir = f"/Users/connorparish/code/hindsight_parsing/data/runs/scrolling_entity/scrolling_entity_{current_time}"
make_dir(model_run_dir)

In [32]:
results = model.train(data=config_f, epochs=20, device="mps", rect=True, 
                      augment=False, close_mosaic=0, batch=15, scale=0, translate=0,
                       name="scrolling_entity", 
                       save_dir=model_run_dir)

New https://pypi.org/project/ultralytics/8.2.93 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.91 🚀 Python-3.10.14 torch-2.5.0.dev20240625 MPS (Apple M3 Max)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04_scrolling_entity/train_config.yaml, epochs=20, time=None, patience=100, batch=15, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=scrolling_entity, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize

train: Scanning /Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04_scrolling_entity/labels... 75 images, 3 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<00:00, 2973.36it/s]

train: New cache created: /Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04_scrolling_entity/labels.cache
WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False



val: Scanning /Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04_scrolling_entity/labels.cache... 75 images, 3 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<?, ?it/s]


Plotting labels to runs/detect/scrolling_entity/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.00046875), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/scrolling_entity
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

WARNING ⚠️ NMS time limit 3.500s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:27<00:00,  9.32s/it]


                   all         75        203      0.932      0.975      0.988      0.852

20 epochs completed in 0.185 hours.
Optimizer stripped from runs/detect/scrolling_entity/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/scrolling_entity/weights/best.pt, 52.0MB

Validating runs/detect/scrolling_entity/weights/best.pt...
Ultralytics YOLOv8.2.91 🚀 Python-3.10.14 torch-2.5.0.dev20240625 MPS (Apple M3 Max)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:29<00:00,  9.71s/it]


                   all         75        203      0.932      0.975      0.988      0.853
Speed: 0.4ms preprocess, 81.5ms inference, 0.0ms loss, 39.0ms postprocess per image
Results saved to runs/detect/scrolling_entity
